<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/AGENTIC-MODULE3-MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# MLP Incremental Training Agent - Your existing MLP code separated
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')

class MLPIncrementalAgent:
    """
    MLP Incremental Training Agent
    Uses your existing MLP training code for continuous learning
    """

    def __init__(self, output_dir='/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/'):
        self.output_dir = output_dir
        self.model = None
        self.transformer = StandardScaler()
        self.is_fitted = False

    def load_synthetic_data(self, data_path=None, windows_path=None):
        """Load pre-generated synthetic data"""
        if data_path is None:
            data_path = f'{self.output_dir}generated-data2.npy'
        if windows_path is None:
            windows_path = f'{self.output_dir}generated-data-true-window2.npy'

        print("Loading synthetic data...")
        x = np.load(data_path)
        y = np.load(windows_path)
        print(f"Loaded: X={x.shape}, y={y.shape}")
        return x, y

    def remove_outliers(self, x, y, contamination=0.4):
        """Remove outliers - your code"""
        print("Removing outliers...")
        iso = IsolationForest(contamination=contamination)
        yhat = iso.fit_predict(x)
        mask = yhat != -1
        x_clean, y_clean = x[mask, :], y[mask]
        print(f"Removed {np.sum(yhat == -1)} outliers. Remaining: {x_clean.shape[0]} samples")
        return x_clean, y_clean

    def prepare_data(self, x, y, test_size=0.1):
        """Prepare and scale data - your code"""
        print("Preparing data...")

        # Transform target - your code
        y_transformed = self.transformer.fit_transform(y.reshape(-1,1)).flatten()

        # Split data - your code
        x_train, x_test, y_train, y_test = train_test_split(
            x, y_transformed, test_size=test_size, random_state=42
        )

        print(f"Train: {x_train.shape}, Test: {x_test.shape}")
        return x_train, x_test, y_train, y_test

    def build_mlp(self):
        """Build MLP model - your exact architecture"""
        print("Building MLP...")

        model = Sequential()
        model.add(Dense(64, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(1))

        optimizr = keras.optimizers.Adam(learning_rate=0.0003, clipnorm=1)
        model.compile(loss='mean_squared_error', optimizer=optimizr, metrics=['mean_squared_error'])

        return model

    def train_initial_mlp(self, x_train, y_train, epochs=1000, batch_size=64):
        """Train MLP from scratch - your code"""
        print("Training initial MLP...")

        # Build model
        self.model = self.build_mlp()

        # Checkpoint callback - your code
        checkpoint_path = f"{self.output_dir}ckp3.weights.h5"
        cp_callback = ModelCheckpoint(
            checkpoint_path,
            monitor='loss',
            save_best_only=True,
            save_weights_only=True
        )

        # Initial build - your code
        self.model.fit(x_train, y_train, epochs=1, batch_size=32)
        self.model.load_weights(checkpoint_path)

        # Early stopping - your code
        es = EarlyStopping(
            patience=20,
            verbose=1,
            min_delta=0.0001,
            monitor='loss',
            mode='min',
            restore_best_weights=True
        )

        # Main training - your code
        history = self.model.fit(
            x_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.1,
            callbacks=[cp_callback]
        )

        self.is_fitted = True
        print("Initial MLP training complete!")
        return history

    def train_incremental_mlp(self, x_new, y_new, epochs=100, learning_rate_factor=0.1):
        """Incremental training on new data"""
        if not self.is_fitted:
            raise ValueError("Must train initial model first!")

        print("Starting incremental training...")

        # Transform new target data
        y_new_transformed = self.transformer.transform(y_new.reshape(-1,1)).flatten()

        # Reduce learning rate for incremental training
        current_lr = self.model.optimizer.learning_rate
        new_lr = current_lr * learning_rate_factor
        self.model.optimizer.learning_rate = new_lr
        print(f"Reduced learning rate to {new_lr}")

        # Incremental training
        history = self.model.fit(
            x_new, y_new_transformed,
            epochs=epochs,
            batch_size=64,
            validation_split=0.1,
            verbose=1
        )

        print("Incremental training complete!")
        return history

    def evaluate_model(self, x_test, y_test, title="Model Evaluation"):
        """Evaluate model - your code"""
        print(f"Evaluating model: {title}")

        # Transform test targets
        if len(y_test.shape) == 1:
            y_test_transformed = self.transformer.transform(y_test.reshape(-1,1)).flatten()
        else:
            y_test_transformed = y_test.flatten()

        # Predict
        y_pred_raw = self.model.predict(x_test)
        y_test_pred = self.transformer.inverse_transform(y_pred_raw)
        y_test_true = self.transformer.inverse_transform(y_test_transformed.reshape(-1,1)).flatten()

        # Calculate R2 - your code
        score = r2_score(y_test_true, y_test_pred)
        print(f"R2 score: {score:.4f}")

        # Plot results - your code
        plt.figure(figsize=(10, 6))
        plt.plot(y_test_true[100:200], color='red', label='Real data')
        plt.plot(y_test_pred[100:200], color='blue', label='Predicted data')
        plt.title(f'{title} - R2: {score:.4f}')
        plt.legend()
        plt.show()

        return score, y_test_pred, y_test_true

    def save_model(self, model_name="METROPM_MLP_model"):
        """Save model - your code"""
        if self.model is None:
            raise ValueError("No model to save!")

        model_path = f'{self.output_dir}{model_name}.keras'
        self.model.save(model_path)
        print(f"Model saved to: {model_path}")
        return model_path

    def load_model(self, model_path):
        """Load existing model"""
        print(f"Loading model from: {model_path}")
        self.model = keras.models.load_model(model_path)
        self.is_fitted = True
        print("Model loaded successfully!")

    def run_initial_training(self, remove_outliers=True, contamination=0.4):
        """
        Complete initial training pipeline - your code flow
        """
        print("="*60)
        print("STARTING INITIAL MLP TRAINING")
        print("="*60)

        # Step 1: Load synthetic data
        x, y = self.load_synthetic_data()

        # Step 2: Remove outliers (optional)
        if remove_outliers:
            x, y = self.remove_outliers(x, y, contamination)

        # Step 3: Prepare data
        x_train, x_test, y_train, y_test = self.prepare_data(x, y)

        # Step 4: Train model
        history = self.train_initial_mlp(x_train, y_train)

        # Step 5: Evaluate
        train_score, _, _ = self.evaluate_model(x_train, y_train, "Training Evaluation")
        test_score, y_pred, y_true = self.evaluate_model(x_test, y_test, "Test Evaluation")

        # Step 6: Save model
        model_path = self.save_model("initial_mlp_model")

        print("="*60)
        print("INITIAL MLP TRAINING COMPLETE!")
        print(f"Train R2: {train_score:.4f}")
        print(f"Test R2: {test_score:.4f}")
        print("="*60)

        return {
            'model': self.model,
            'model_path': model_path,
            'train_score': train_score,
            'test_score': test_score,
            'history': history,
            'test_predictions': y_pred,
            'test_true': y_true
        }

    def run_incremental_update(self, new_x_data, new_y_data, epochs=100, model_name="updated_mlp"):
        """
        Incremental update with new data
        This is the main agent interface for continuous learning
        """
        print("="*60)
        print("STARTING INCREMENTAL MLP UPDATE")
        print("="*60)

        # Incremental training
        history = self.train_incremental_mlp(new_x_data, new_y_data, epochs=epochs)

        # Evaluate on new data
        score, y_pred, y_true = self.evaluate_model(
            new_x_data, new_y_data, "Incremental Evaluation"
        )

        # Save updated model
        model_path = self.save_model(model_name)

        print("="*60)
        print("INCREMENTAL MLP UPDATE COMPLETE!")
        print(f"Updated R2: {score:.4f}")
        print("="*60)

        return {
            'model': self.model,
            'model_path': model_path,
            'score': score,
            'history': history,
            'predictions': y_pred,
            'true_values': y_true
        }

# Usage Examples
if __name__ == "__main__":

    # Initialize agent
    agent = MLPIncrementalAgent()

    # ===== INITIAL TRAINING (One time) =====
    print("=== INITIAL TRAINING ===")
    initial_results = agent.run_initial_training()

    # ===== INCREMENTAL UPDATES (When new data arrives) =====
    print("\n=== INCREMENTAL UPDATES ===")

    # Simulate new data arrival
    # In practice, this would be new synthetic data generated from VAE
    x_new, y_new = agent.load_synthetic_data()  # Load some new data
    x_new_sample = x_new[:1000]  # Take a sample as "new" data
    y_new_sample = y_new[:1000]

    # Incremental update
    update_results = agent.run_incremental_update(
        x_new_sample,
        y_new_sample,
        epochs=100,
        model_name="updated_mlp_v1"
    )

    # Can continue with more updates
    for i in range(3):
        print(f"\n--- Additional Update {i+1} ---")
        start_idx = (i+1) * 1000
        end_idx = (i+2) * 1000

        update_results = agent.run_incremental_update(
            x_new[start_idx:end_idx],
            y_new[start_idx:end_idx],
            epochs=50,
            model_name=f"updated_mlp_v{i+2}"
        )

        print(f"Update {i+1} complete. R2: {update_results['score']:.4f}")

Epoch 1/1000
4430/4430 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 0.8964 - mean_squared_error: 0.8964 - val_loss: 0.7946 - val_mean_squared_error: 0.7946
Epoch 2/1000
4430/4430 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 0.8044 - mean_squared_error: 0.8044 - val_loss: 0.7901 - val_mean_squared_error: 0.7901
Epoch 3/1000
4430/4430 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 0.7842 - mean_squared_error: 0.7842 - val_loss: 0.7714 - val_mean_squared_error: 0.7714
Epoch 4/1000
4430/4430 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 0.7728 - mean_squared_error: 0.7728 - val_loss: 0.7566 - val_mean_squared_error: 0.7566
Epoch 5/1000
4430/4430 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 0.7643 - mean_squared_error: 0.7643 - val_loss: 0.7566 - val_mean_squared_error: 0.7566
Epoch 6/1000
4430/4430 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 0.7626 - mean_squared_error: 0.7626 - val_loss: 0.7513 - val_mean_squared_error: 0.7513
Epoch 7/1000
4430/4430 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 0.7563 - mean_squared_e

In [ ]:
from google.colab import drive
drive.mount('/content/drive')